In [2]:
import numpy as np
import pandas as pd

In [3]:
# Load data
Data = pd.read_csv("Life Expectancy Data.csv")
# Grab column names
cols = Data.columns

In [4]:
# Check the percentage of None data for each colume
Data.isna().sum()/len(Data)
# We have a bunch of missing data to deal with!

Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.003404
Adult Mortality                    0.003404
infant deaths                      0.000000
Alcohol                            0.066031
percentage expenditure             0.000000
Hepatitis B                        0.188223
Measles                            0.000000
 BMI                               0.011572
under-five deaths                  0.000000
Polio                              0.006467
Total expenditure                  0.076923
Diphtheria                         0.006467
 HIV/AIDS                          0.000000
GDP                                0.152485
Population                         0.221920
 thinness  1-19 years              0.011572
 thinness 5-9 years                0.011572
Income composition of resources    0.056841
Schooling                          0.055480
dtype: float64

In [5]:
# Drop rows that has no "life expectancy" value since very few are missing it, 
# and it is the most important value to have for our analysis
Data = Data[~Data["Life expectancy "].isna()]

# Also drop rows that are missing values in cloumns with more than 95% of the values 
non_perc = (Data.isna().sum()/len(Data)).values
less5Cols = non_perc < 0.05
for col in cols[less5Cols]:
    if Data[col].isna().sum()/len(Data) > 1e-3:
        Data = Data[~Data[col].isna()]

In [6]:
# replace Developing/Developed column with boolean values
Data = Data.replace("Developing", False)
Data = Data.replace("Developed", True)

C:\Users\danie\AppData\Local\Temp\ipykernel_17592\4202270222.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data = Data.replace("Developed", True)


In [7]:
# Fix the bad indexing created by dropping rows
Data = Data.reset_index()
Data.drop('index', axis=1, inplace=True)
# Find all the columns that still have missing values
non_perc = (Data.isna().sum()/len(Data)).values
ColsWithNone = non_perc > 0
new_Data = Data.copy()
# Get average values for each column by country
country_avgs = Data.groupby("Country").mean()
# If every entry in a column for a country is missing, fill them all with the mean in that column from every country
for col in country_avgs.columns:
    country_avgs.fillna({col : country_avgs[col].mean()}, inplace=True)
# Fill every missing value with the country's average.
for col in cols[ColsWithNone]:
    # Get the row of values we should be filling with
    avgs = country_avgs[col]
    # Go through missing values and replace with the value from the country's average
    indx = Data[col].isna()
    for i in np.where(indx)[0]:
        new_Data.loc[i,col] = avgs[Data["Country"][i]]

In [8]:
# Save the cleaned data
new_Data.to_csv("Life Expectancy Data Cleaned.csv")